In [240]:
import pandas as pd
import numpy as np
import os
import keras
from datetime import timedelta, datetime

ImportError: cannot import name 'strptime' from 'datetime' (/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/datetime.py)

In [208]:
#os.chdir("src") #used to reset to original working directory

FileNotFoundError: [Errno 2] No such file or directory: 'src'

In [287]:
model = keras.Sequential()
model.add(keras.layers.LSTM(8,input_shape= (3,1)))
model.add(keras.layers.Dense(23))
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 8)                 320       
                                                                 
 dense_13 (Dense)            (None, 23)                207       
                                                                 
Total params: 527
Trainable params: 527
Non-trainable params: 0
_________________________________________________________________


In [218]:
os.chdir("../") #Used to go out into the cmpe-351-group-1 folder to access the data without changing directories multiple times

In [288]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [194]:
crypto_dict = {
    "bitcoin":"btc",
    "avalanche":"avax",
    "doge":"doge",
    "ethereum":"eth",
    "solana":"sol"
}

In [ ]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
            +timedelta(hours=t.minute//30))

In [507]:
##Dates is an array of tuples consisting of month and day as numbers
def get_dataset(crypto,start_month,start_day,end_month,end_day):
    twitter_df = pd.read_csv(f'data/processed/twitter/{crypto}_sentiment.csv')
    try:
        bit_prices = pd.read_csv(f'data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}_minute.csv')
    except:
        bit_prices = pd.read_csv(f'data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}.csv')
    bit_prices["time"] = pd.to_datetime(bit_prices['time'],unit='s')
    price_dict = bit_prices.set_index("time")["open"].to_dict()
    twitter_df["created_at"] = twitter_df["created_at"].apply(lambda x: pd.to_datetime(x).tz_localize(None))
    twitter_df["created_at"] = twitter_df["created_at"].apply(lambda x: hour_rounder(x))
    twitter_df["future_date"] = twitter_df["created_at"].apply(lambda x: x+timedelta(hours=23))

    twitter_df = twitter_df[twitter_df["future_date"].isin(bit_prices["time"])] ##Filter out dates for which we don't have price data

    #print(len(twitter_df))

    twitter_df = twitter_df[twitter_df["created_at"].isin(bit_prices["time"])] ##Filter out dates for which we don't have price data


    twitter_df["price"] = twitter_df["created_at"].apply(lambda x: price_dict[x])
    for i in range(1,24):
        twitter_df[f"price_{i}hours"] = twitter_df["created_at"].apply(lambda x: price_dict[x+timedelta(hours=i)])
    X = twitter_df[["retweet_count","vader_sentiment_compound","price"]]
    y = twitter_df[[f"price_{i}hours" for i in range(1,24)]]
    return twitter_df,X,y


In [508]:
df_avax,X_avax,y_avax = get_dataset("avalanche","03","05","03","11")
df_avax2, X_avax2, y_avax2 = get_dataset("avalanche","03","28","04","04")
df_avax3, X_avax3, y_avax3 = get_dataset("avalanche","03","11","03","12")
df_avax4, X_avax4, y_avax4 = get_dataset("avalanche","04","04","04","05")

df_avax = df_avax.append(df_avax2)
df_avax = df_avax.append(df_avax3)
df_avax = df_avax.append(df_avax4)

X_avax = X_avax.append(X_avax2)
X_avax = X_avax.append(X_avax3)
X_avax = X_avax.append(X_avax4)

y_avax = y_avax.append(y_avax2)
y_avax = y_avax.append(y_avax3)
y_avax = y_avax.append(y_avax4)


In [509]:
df_btc,X_btc,y_btc = get_dataset("bitcoin","03","05","03","11")
df_btc2, X_btc2, y_btc2 = get_dataset("bitcoin","03","28","04","04")
df_btc3, X_btc3, y_btc3 = get_dataset("bitcoin","03","11","03","12")
df_btc4, X_btc4, y_btc4 = get_dataset("bitcoin","04","04","04","05")

df_btc = df_btc.append(df_btc2)
df_btc = df_btc.append(df_btc3)
df_btc = df_btc.append(df_btc4)

X_btc = X_btc.append(X_btc2)
X_btc = X_btc.append(X_btc3)
X_btc = X_btc.append(X_btc4)

y_btc = y_btc.append(y_btc2)
y_btc = y_btc.append(y_btc3)
y_btc = y_btc.append(y_btc4)

In [457]:
df_doge,X_doge,y_doge = get_dataset("doge","03","05","03","11")
df_doge2,X_doge2,y_doge2 = get_dataset("doge","03","28","04","04")
df_doge3,X_doge3,y_doge3 = get_dataset("doge","03","11","03","12")
df_doge4,X_doge4,y_doge4 = get_dataset("doge","04","04","04","05")

df_doge = df_doge.append(df_doge2)
df_doge = df_doge.append(df_doge3)
df_doge = df_doge.append(df_doge4)

X_doge = X_doge.append(X_doge2)
X_doge = X_doge.append(X_doge3)
X_doge = X_doge.append(X_doge4)

y_doge = y_doge.append(y_doge2)
y_doge = y_doge.append(y_doge3)
y_doge = y_doge.append(y_doge4)

In [513]:
df_eth,X_eth,y_eth = get_dataset("ethereum","03","05","03","11")
df_eth2, X_eth2, y_eth2 = get_dataset("ethereum","03","28","04","04")
df_eth3,X_eth3,y_eth3 = get_dataset("ethereum","03","11","03","12")
df_eth4, X_eth4, y_eth4 = get_dataset("ethereum","04","04","04","05")

df_eth = df_eth.append(df_eth2)
df_eth = df_eth.append(df_eth3)
df_eth = df_eth.append(df_eth4)



X_eth = X_eth.append(X_eth2)
X_eth = X_eth.append(X_eth3)
X_eth = X_eth.append(X_eth4)

y_eth = y_eth.append(y_eth2)
y_eth = y_eth.append(y_eth3)
y_eth = y_eth.append(y_eth4)

In [514]:
df_sol,X_sol,y_sol = get_dataset("solana","03","05","03","11")
df_sol2, X_sol2, y_sol2 = get_dataset("solana","03","28","04","04")
df_sol3,X_sol3,y_sol3 = get_dataset("solana","03","11","03","12")
df_sol4, X_sol4, y_sol4 = get_dataset("solana","04","04","04","05")

df_sol = df_sol.append(df_sol2)
df_sol = df_sol.append(df_sol3)
df_sol = df_sol.append(df_sol4)


X_sol = X_sol.append(X_sol2)
X_sol = X_sol.append(X_sol3)
X_sol = X_sol.append(X_sol4)

y_sol = y_sol.append(y_sol2)
y_sol = y_sol.append(y_sol3)
y_sol = y_sol.append(y_sol4)


In [515]:
def estimate_price(model,date,df):
    date_23_hours = date-timedelta(hours=23)
    date_1_hours = date-timedelta(hours=1)
    df_useful = df[(df["created_at"]<=date_1_hours) & (df["created_at"]>=date_23_hours)]
    if len(df_useful)==0:
        return ""
    else:
        df_useful["idx"] = df_useful["created_at"].apply(lambda x: round((date-x).seconds/60/60))
        X = df_useful[["retweet_count","vader_sentiment_compound","price"]]
        y_hat = model.predict_on_batch(X)
        idx = df_useful["idx"].values
        vals = []
        for i in range(len(y_hat)):
            vals.append(y_hat[i][idx[i]-1])
        return np.mean(vals)

In [ ]:
bit_model = model.fit(X_btc,y_btc,epochs=1000,batch_size=10)

Epoch 1/1000
163/163 [==============================] - 0s 3ms/step - loss: 2016382336.0000 - mean_squared_error: 2016382336.0000
Epoch 2/1000
163/163 [==============================] - 0s 3ms/step - loss: 2016310144.0000 - mean_squared_error: 2016310144.0000
Epoch 3/1000
163/163 [==============================] - 1s 4ms/step - loss: 2016260736.0000 - mean_squared_error: 2016260736.0000
Epoch 4/1000
163/163 [==============================] - 1s 3ms/step - loss: 2016219520.0000 - mean_squared_error: 2016219520.0000
Epoch 5/1000
163/163 [==============================] - 1s 3ms/step - loss: 2016181248.0000 - mean_squared_error: 2016181248.0000
Epoch 6/1000
163/163 [==============================] - 0s 3ms/step - loss: 2016147200.0000 - mean_squared_error: 2016147200.0000
Epoch 7/1000
163/163 [==============================] - 0s 3ms/step - loss: 2016114688.0000 - mean_squared_error: 2016114688.0000
Epoch 8/1000
163/163 [==============================] - 1s 4ms/step - loss: 2016083840.000

In [ ]:
eth_model = model.fit(X_eth,y_eth,epochs=1000,batch_size=10)

In [ ]:
sol_model = model.fit(X_sol,y_sol,epochs=1000,batch_size=10)

In [ ]:
doge_model = model.fit(X_doge,y_doge,epochs=750,batch_size=10)

In [ ]:
avax_model = model.fit(X_avax,y_avax,epochs=1000,batch_size=10)

In [393]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)*24):
        yield start_date + timedelta(hours=n)

In [ ]:
def make_prediction(start_date, end_date, model,df):
    pred = {}
    for date in daterange(start_date,end_date):
        val = estimate_price(model,date,df)
        if val!='':
            pred[date] = val
    return pred